In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import product
from datetime import date
from path import Path
import pandas as pd
import random

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING


DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbol_groups": [["equity_index"]],
#     "symbol_groups": [[x] for x in symbol_groups],
#     "symbol_groups": [['equity_index'], ['interest_rate']],
#     "symbols": [['@NQ#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
    "binarize_params": [10, 25, 50, 100],
#     "binarize_params": [10, 25, 100],
#     "binarize_params": [50],
    "alpha": ["none"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
#     "classifier": ["random_forest", "xgboost", "lgbm", "knn", "dummy"],
#     "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
#     "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
    "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
#     "num_threads": [1], # how many parallel hyperopti 
#     "n_jobs": [1], # how many parallel infer
    "feat_imp_n_estimators": [500],
    "feat_imp_cv": [5],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [False],
#     "optimize_hypers": [False],
#     "reuse_hypers": [False],
#     "load_from_disk": [False],
    "skip_feature_imp": [True],
#     "pca_transform": [False],
}

c:\users\doda\anaconda3\envs\fincl\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\doda\anaconda3\envs\fincl\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
2020-02-20 15:05:52,368 Loaded backend module://ipykernel.pylab.backend_inline version unknown.


49
c_s 1


In [3]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 10,
  'alpha': 'none',
  'features': [{'name': 'ffd', 'd': 0.3},
   {'name': 'log_ret', 'periods': 25},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'rolling_stdev', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'lag',
    'lag': 25,
    'symbol': {'name': 'ewm_mean',
     'window': 25,
     'symbol': {'name': 'volratio'}}},
   {'name': 'rolling_stdev',
    'window': 25,
    'symbol': {'name': 'log_ret',
     'symbol': {'name': 'make_bars',
      'type_': 'time',
      'size': 60,
      'symbol': 'VIX.XO'}}},
   {'name': '

In [4]:
len(datas)

4

In [5]:
# [x['features'][0] for x in datas]

In [6]:
from mlbt.multiprocess import process_jobs

In [7]:
num_threads = max(1, min(32, len(datas)) - 0)
num_threads
      

4

In [8]:
res = process_jobs(datas, num_threads=num_threads)

2020-02-20 15:06:57.298276 100.0% run_bt done after 1.04 minutes. Remaining 0.0 minutes.


In [24]:
for data in datas:
    run_bt(**data)

2020-02-20 14:51:04,334 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'vol_estimate': 100, 'downsampling': 'cusum', 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'ffd', 'd': 0.3}, {'name': 'log_ret', 'periods': 10}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'rolling_stdev', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'lag', 'lag': 10, 'symbol': {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}}, {'name': 'rolling_stdev', 'window': 10, 'symbol': {'name': 'log_ret', 'symbol': {'name': 'make_bars', 'type_': 'time', 'siz

2020-02-20 14:51:04,462 @NKD#C: Have 73050 bars and 33604 binarized events
2020-02-20 14:51:04,472 @NE#C: Have 45459 bars and 1156 binarized events
2020-02-20 14:51:04,483 @RP#C: Have 33820 bars and 1274 binarized events
2020-02-20 14:51:04,494 BD#C: Have 68392 bars and 32049 binarized events
2020-02-20 14:51:04,508 EZ#C: Have 110422 bars and 42748 binarized events
2020-02-20 14:51:04,521 XG#C: Have 71217 bars and 34548 binarized events
2020-02-20 14:51:04,533 EX#C: Have 74018 bars and 34509 binarized events
2020-02-20 14:51:04,544 @YM#C: Have 39311 bars and 18877 binarized events
2020-02-20 14:51:04,557 @BO#C: Have 93715 bars and 44853 binarized events
2020-02-20 14:51:04,570 @C#C: Have 80542 bars and 36603 binarized events
2020-02-20 14:51:04,581 @FV#C: Have 53497 bars and 24547 binarized events
2020-02-20 14:51:04,593 @TY#C: Have 55083 bars and 25225 binarized events
2020-02-20 14:51:04,603 @TU#C: Have 39910 bars and 15964 binarized events
2020-02-20 14:51:04,614 @UB#C: Have 33182 b

2020-02-20 14:51:35,044 Joined 74 features into (50590, 74) shape
2020-02-20 14:51:35,045 @QG#C: Feature engineering for 74 features
2020-02-20 14:51:35,896 Joined 74 features into (186060, 74) shape
2020-02-20 14:51:35,896 @QM#C: Feature engineering for 74 features
2020-02-20 14:51:36,655 Joined 74 features into (96774, 74) shape
2020-02-20 14:51:36,656 QGC#C: Feature engineering for 74 features
2020-02-20 14:51:37,373 Joined 74 features into (39571, 74) shape
2020-02-20 14:51:37,374 QHG#C: Feature engineering for 74 features
2020-02-20 14:51:38,107 Joined 74 features into (67132, 74) shape
2020-02-20 14:51:38,108 QSI#C: Feature engineering for 74 features
2020-02-20 14:51:38,837 Joined 74 features into (59640, 74) shape
2020-02-20 14:51:38,838 @AD#C: Get bins and feature imps
c:\users\doda\anaconda3\envs\fincl\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

2020-02-20 14:51:46,624 Dropped label 0.0 0.004821524236591996
2020-02-20 14:51:46,990 bars (106975, 9), events (50449, 2), feats (106975, 74), bins (50156, 3), X_all (48890, 74), X_train (24445, 74)
2020-02-20 14:51:46,991 @MME#C: Get bins and feature imps
2020-02-20 14:51:47,006 Dropped label 0.0 0.001221230624706435
2020-02-20 14:51:47,241 bars (22420, 9), events (10695, 2), feats (22420, 74), bins (10632, 3), X_all (9290, 74), X_train (4645, 74)
2020-02-20 14:51:47,242 @CC#C: Get bins and feature imps
2020-02-20 14:51:47,258 Dropped label 0.0 0.004170113041080783
2020-02-20 14:51:47,564 bars (61619, 9), events (29066, 2), feats (61619, 74), bins (28895, 3), X_all (27627, 74), X_train (13813, 74)
2020-02-20 14:51:47,564 @CT#C: Get bins and feature imps
2020-02-20 14:51:47,582 Dropped label 0.0 0.0016760361744474319
2020-02-20 14:51:47,875 bars (61020, 9), events (28689, 2), feats (61020, 74), bins (28591, 3), X_all (27220, 74), X_train (13610, 74)
2020-02-20 14:51:47,876 @OJ#C: Get 

2020-02-20 14:51:52,779 @SB#C [(16426, 5), (16426, 74), (16426,), (16426, 5), (16426, 74), (16426,)]
2020-02-20 14:51:52,779 GAS#C [(5891, 5), (5891, 74), (5891,), (5891, 5), (5891, 74), (5891,)]
2020-02-20 14:51:52,779 LF#C [(13964, 5), (13964, 74), (13964,), (13964, 5), (13964, 74), (13964,)]
2020-02-20 14:51:52,779 LG#C [(10585, 5), (10585, 74), (10585,), (10586, 5), (10586, 74), (10586,)]
2020-02-20 14:51:52,780 QCL#C [(14532, 5), (14532, 74), (14532,), (14532, 5), (14532, 74), (14532,)]
2020-02-20 14:51:52,780 QHO#C [(3953, 5), (3953, 74), (3953,), (3954, 5), (3954, 74), (3954,)]
2020-02-20 14:51:52,780 QNG#C [(13780, 5), (13780, 74), (13780,), (13781, 5), (13781, 74), (13781,)]
2020-02-20 14:51:52,781 QPA#C [(7272, 5), (7272, 74), (7272,), (7272, 5), (7272, 74), (7272,)]
2020-02-20 14:51:52,781 QPL#C [(11709, 5), (11709, 74), (11709,), (11710, 5), (11710, 74), (11710,)]
2020-02-20 14:51:52,781 @QG#C [(19365, 5), (19365, 74), (19365,), (19366, 5), (19366, 74), (19366,)]
2020-02-20

In [61]:
%debug

> c:\users\doda\anaconda3\envs\fincl\lib\site-packages\timeseriescv\cross_validation.py(53)split()
     51             raise ValueError('X and y must have the same index')
     52         if (X.index == pred_times.index).sum() != len(pred_times):
---> 53             raise ValueError('X and pred_times must have the same index')
     54         if (X.index == eval_times.index).sum() != len(eval_times):
     55             raise ValueError('X and eval_times must have the same index')

ipdb> exit


In [62]:
1

1